### Agente con consultas a una Base de Datos

Definimos la ruta para entrar a la base de datos de sqlite

In [1]:
from pathlib import Path

base_dir = Path.cwd()
database_dir = base_dir / "Databases"

Definimos el LLM

In [5]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

openai_model = os.getenv("LLM_MODEL")

# Crear una instancia del modelo ChatOpenAI
llm = ChatOpenAI(model= openai_model, temperature=0.5)

Definimos la herramienta que recuperará los productos de la base de datos

In [6]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Databases/Productos.db",)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Categories', 'Products']
